In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import os, os.path
import pathlib
from pandas import *
from pathlib import Path
import face_recognition
from tqdm import tqdm


# train
folder_path = ' '
processed_folder_path = ' '
test_path = ' '
model_dir = ' '
csv_data = read_csv(folder_path+"train.csv")

#==================================

# train_small
# folder_path = 
# processed_folder_path = 
# test_path = 
# model_dir = 
# csv_data = read_csv(folder_path+"train_small.csv")


batch_size = 32
img_height = 224
img_width = 224
rescale = (img_height, img_width)

# reading CSV file
category = csv_data['Category'].tolist()
file_name = csv_data['File Name'].tolist()

def rescale_image(file_path, rescale): # rescale image to a specfic size 
    # Load the image
    image = cv2.imread(file_path, -1)
    
    if image is None:
        return None    
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image= cv2.resize(image, rescale)
    #image = image.astype('float32')
    #image /= 255.0

    return image

def extracter_opencv(file_path, rescale): # extract face using OpenCV [X]
    multi_indicator = 0
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    # Read the input image
    img = cv2.imread(file_path)
    if (img is None) or (np.shape(img)[0] == 0):
        return None, multi_indicator
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.01, 1)
    # Crop
    if np.shape(faces)[0] > 1: # if more than one person detected
        face = np.zeros((rescale[0],rescale[1],np.shape(faces)[0]))
        for i in range(0,np.shape(faces)[0]):
            x, y, width, height = faces[i]
            x, y = abs(x), abs(y)
            x2, y2 = x + width, y + height
            face_temp = gray[y:y2, x:x2]
            face_temp = cv2.resize(face_temp, rescale)
            face[:,:,i] = face_temp
        multi_indicator = 1

    elif np.shape(faces)[0] == 0: # No person
        face = None

    else: # only one person detected
        face = np.zeros((rescale[0],rescale[1],np.shape(faces)[0]))
        x, y, width, height = faces[0]
        x, y = abs(x), abs(y)
        x2, y2 = x + width, y + height
        face = gray[y:y2, x:x2]
        face = cv2.resize(face, rescale)

    return face, multi_indicator

def extracter_mtcnn(file_path, rescale): # extract face using MTCNN [X]

    multi_indicator = 0

    # load image
    image = cv2.imread(file_path,-1)
    if (image is None) or (np.shape(image)[0] == 0):
        return None, multi_indicator

    face = MTCNN().detect_faces(image)

    # No face
    if not face:
        print("No face detected")
        return None, multi_indicator

    image = cv2.imread(file_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    # cv2.imshow('face', image)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    gray = np.zeros((rescale[0],rescale[1],3,len(face)))
    if len(face) > 1:
        multi_indicator = 1

    for i in range(0,len(face)):
        # Extract the face bounding box
        x, y, width, height = face[i]['box']
        x, y = abs(x), abs(y)
        x2, y2 = x + width, y + height

        image_temp = cv2.resize(image[y:y2, x:x2,:], rescale)
        gray[:,:,:,i] = image_temp
        #gray[:,:,:,i] = cv2.cvtColor(image_temp, cv2.COLOR_BGR2RGB)
        
        # temp = cv2.cvtColor(image_temp, cv2.COLOR_BGR2RGB)
        # print(np.shape(temp))
        # Crop, resize, and normalize the face
        #gray[:,:,:,i] = cv2.resize(image[y:y2, x:x2, :], rescale)

    return gray, multi_indicator

def extracter_fcrg(file_path, rescale): # extract face using face recognition 

    multi_indicator = 0

    # load image
    image = cv2.imread(file_path,-1)
    if (image is None) or (np.shape(image)[0] == 0):
        print("None image")
        return None, multi_indicator

    input_img = face_recognition.load_image_file(file_path)
    face = face_recognition.face_locations(input_img)

    # No face
    if not face:
        print("No face detected")
        return None, multi_indicator

    image = cv2.imread(file_path)
    gray = np.zeros((rescale[0],rescale[1],3,len(face)))
    if len(face) > 1:
        multi_indicator = 1
        return None, multi_indicator #*********************

    for i in range(0,len(face)):
        # Extract the face bounding box
        # x, y, width, height = face[i] 
        # x, y = abs(x), abs(y)
        # x2, y2 = x + width, y + height

        # top, right, bottom, left (64, 201, 219, 46)
        #left, right, top, bottom = face[i]
        top, right, bottom, left = face[i]
        # image_temp = cv2.resize(image[y:y2, x:x2,:], rescale)
        image_temp = cv2.resize(image[top:bottom, left:right,:], rescale)
        gray[:,:,:,i] = image_temp
        # cv2.imshow('face', image_temp)
        # cv2.waitKey()
        # cv2.destroyAllWindows()

    return gray, multi_indicator

def extracter_fcrg_prd(file_path, rescale): # extract face using face recognition for prediction

    multi_indicator = 0

    # load image
    image = cv2.imread(file_path,-1)
    if (image is None) or (np.shape(image)[0] == 0):
        print("None image")
        return None, multi_indicator

    input_img = face_recognition.load_image_file(file_path)
    face = face_recognition.face_locations(input_img)

    # No face
    if not face:
        print("No face detected")
        return None, multi_indicator

    image = cv2.imread(file_path)
    gray = np.zeros((rescale[0],rescale[1],3,len(face)))
    if len(face) > 1:
        multi_indicator = 1

    for i in range(0,len(face)):
        # Extract the face bounding box
        # x, y, width, height = face[i] 
        # x, y = abs(x), abs(y)
        # x2, y2 = x + width, y + height

        # top, right, bottom, left (64, 201, 219, 46)
        #left, right, top, bottom = face[i]
        top, right, bottom, left = face[i]
        # image_temp = cv2.resize(image[y:y2, x:x2,:], rescale)
        image_temp = cv2.resize(image[top:bottom, left:right,:], rescale)
        gray[:,:,:,i] = image_temp
        # cv2.imshow('face', image_temp)
        # cv2.waitKey()
        # cv2.destroyAllWindows()

    return gray, multi_indicator

In [ ]:
#--------------------------------------------
# Image preprocessing (MTCNN): https://medium.com/@saranshrajput/face-detection-using-mtcnn-f3948e5d1acb
#--------------------------------------------

# [X]

In [ ]:
#--------------------------------------------
# Image preprocessing (OpenCV): https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81
#--------------------------------------------

# [X]

In [ ]:
#--------------------------------------------
# Image preprocessing (face_recognition): https://github.com/ageitgey/face_recognition
#--------------------------------------------

for i in range(0,len(category)):
    if i%5 == 0:
        print('Preprocessing... '+str(i)+'/'+str(len(category)))

    try:
        # if folder not eixsts, create folder:
        if not os.path.exists(processed_folder_path+str(category[i])):
            os.makedirs(processed_folder_path+str(category[i]))
            image_count = 1   
        else:
            # count number of files in the folder
            image_count = len([name for name in os.listdir(processed_folder_path+str(category[i])+"\\")])+1

        face2, multi_indicator = extracter_fcrg(folder_path+str(i)+'.jpg', rescale)
        if (face2 is None):
            pass
            # face0 = rescale_image(folder_path+str(i)+'.jpg', rescale)
            # if (face0 is None):
            #     pass
            # else:
            #     # print(np.shape(face0))
            #     face0_image = PIL.Image.fromarray(face0,'RGB')
            #     #face0_image = face0_image.convert("L")
            #     face0_image.save(processed_folder_path+str(category[i])+'\\'+str(image_count)+'.jpg', 'JPEG')
            #     image_count += 1
        elif multi_indicator == 1:
            # for j in range(np.shape(face2)[3]):
            #     temp = np.asarray(face2[:,:,:,j], dtype=np.float32)
            #     cv2.imwrite(processed_folder_path+str(category[i])+'\\'+str(image_count)+'.jpg', temp)
            #     image_count += 1
            pass   
        else:
            temp = np.asarray(face2[:,:,:,0], dtype=np.float32)
            cv2.imwrite(processed_folder_path+str(category[i])+'\\'+str(image_count)+'.jpg', temp)
            image_count += 1
    except:
        pass


In [ ]:
#--------------------------------------------
# Generate training & validation set: https://www.tensorflow.org/tutorials/images/classification
#--------------------------------------------

import matplotlib.pyplot as plt

train_ds = tf.keras.utils.image_dataset_from_directory(
  processed_folder_path,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  processed_folder_path,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print(class_names)

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")


In [ ]:
#--------------------------------------------
# NN setting (GhostFaceNet V2): https://github.com/HamadYA/GhostFaceNets
#                               https://www.tensorflow.org/tutorials/images/classification
#--------------------------------------------

# [X]

# import losses, train, GhostFaceNets

# num_classes = len(class_names)
# checkpoint_path = model_dir+'cp-{epoch:04d}.weights.h5'
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                 #  monitor='val_accuracy',
#                                                 #  mode='max',
#                                                 #  save_best_only=True,
#                                                  verbose=1)


# emb_shape = 512
# basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0.5, emb_shape=emb_shape, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5,num_classes=num_classes)
# basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=True)
# basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)
# # basic_model = tf.keras.models.load_model('ghostnetv2_160_imagenet.h5')

# # sch = [
# #      {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1},
# #      {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
# #  ]
# # #basic_model.compile(optimizer, loss=losses.ArcfaceLoss(scale=32), metrics=['accuracy'])
# # #basic_model.compile(optimizer, sch, metrics=['accuracy'])
# # basic_model.compile(optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# # basic_model.summary()

# normalization_layer = layers.Rescaling(1./255)
# data_augmentation = keras.Sequential(
#    [
#      layers.RandomFlip("horizontal",
#                        input_shape=(img_height,
#                                     img_width,
#                                    3)),
#      layers.RandomRotation(0.3),
#      layers.RandomZoom(0.3),
#    ]
# )

# model = Sequential([
#     data_augmentation,
#     #normalization_layer,
#     basic_model,
#     layers.Dense(emb_shape/2, activation='relu'),
#     layers.Dense(num_classes)
#   ])

# optimizer = tf.keras.optimizers.experimental.SGD(learning_rate=0.1, momentum=0.9)
# model.compile(optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['SparseCategoricalAccuracy'])
# # model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# model.summary()


In [ ]:
#--------------------------------------------
# NN setting (VGGnet): https://github.com/rcmalli/keras-vggface
#                      https://www.tensorflow.org/tutorials/images/classification
#--------------------------------------------
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

checkpoint_path = model_dir+'VGG_cp-{epoch:04d}.weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                #  monitor='val_accuracy',
                                                #  mode='max',
                                                #  save_best_only=True,
                                                 verbose=1)

VGG = VGGFace(model = 'resnet50', include_top = False, input_shape = (224, 224, 3), pooling = 'avg')
VGG.trainable = False

# normalization_layer = layers.Rescaling(1./255)

data_augmentation = keras.Sequential(
   [
     layers.RandomTranslation(height_factor = 0.15, width_factor = 0.15, input_shape = (224, 224, 3)),
     layers.RandomRotation(factor = 0.15),
     layers.RandomFlip(mode = "horizontal"),
     layers.RandomZoom(height_factor = 0.05, width_factor = 0.05),
     layers.RandomContrast(factor = 0.15)
   ]
)

model = Sequential([
    data_augmentation,
    #normalization_layer,
    VGG,
    layers.Flatten(),
    layers.Dense(1024, activation = 'relu'),
    layers.Dropout(0.15), # 0.3?
    layers.Dense(100, activation = 'softmax')
  ])

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),  
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4), # 1e-2
    metrics = ['sparse_categorical_accuracy'])

model.summary()

In [ ]:
#--------------------------------------------
# ! Temporal: load weight
#--------------------------------------------

# model = tf.keras.models.load_model(model_dir +'0331_temp_model.keras')
# model = tf.keras.models.load_model(model_dir +'0401_temp_model.keras')
# model = tf.keras.models.load_model(model_dir +'0402_temp_model_v3.keras')
# model.load_weights(model_dir +'temp.h5')

In [ ]:
#--------------------------------------------
# Training
#--------------------------------------------

epochs=30
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  batch_size=batch_size,
  callbacks=[cp_callback]
)


In [ ]:
#--------------------------------------------
# Save model
#--------------------------------------------

model.save(model_dir+'VGG_final.keras')
# model.save(model_dir+'Ghost_final.keras')

In [ ]:
#--------------------------------------------
# Performance check
#--------------------------------------------

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#--------------------------------------------
# Generate prediction
#--------------------------------------------

#model = tf.keras.models.load_model(model_dir +'epoch_7_transfer_model.keras')
# model = tf.keras.models.load_model(model_dir +'0401_temp_model.keras')

image_count = len([name for name in os.listdir(test_path)])
image_names=os.listdir(test_path)
#img = tf.keras.utils.load_img(test_path, target_size=(img_height, img_width))

if image_count != 4977:
    print("You are doing someting wrong!!!")

for i in range(0,image_count):

    try:
        # detect face from test image
        #face_test, test_indicator = extracter_mtcnn(test_path+filename, rescale)
        face_test, test_indicator = extracter_fcrg_prd(test_path+str(i)+'.jpg', rescale)

        # No face detected, predict from original image
        if (face_test is None):
            img = tf.keras.preprocessing.image.load_img(test_path+str(i)+'.jpg', target_size=(img_height, img_width, 3))
            x=np.expand_dims(img, axis=0)
            predictions = model.predict(x)
            predicted_class = class_names[np.argmax(predictions)]

        # Multiple faces are detected
        elif test_indicator == 1:
            print('multi-face detected')
            for j in range(np.shape(face_test)[3]):
                x=np.expand_dims(face_test[:,:,:,j], axis=0)
                predictions = model.predict(x)
                if j == 0:
                    multi_predictor = predictions[0,np.argmax(predictions)]
                    predicted_class = class_names[np.argmax(predictions)]
                else:
                    temp_predictor = predictions[0,np.argmax(predictions)]
                    if temp_predictor > multi_predictor:
                        multi_predictor = temp_predictor
                        predicted_class = class_names[np.argmax(predictions)]
                        
        # Only one face detected
        else:
            #face_test_image = PIL.Image.fromarray(face_test[:,:,0])
            #x=np.expand_dims(face_test_image, axis=0)
            x=np.expand_dims(face_test[:,:,:,0], axis=0)
            predictions = model.predict(x)
            predicted_class = class_names[np.argmax(predictions)]

        if i == 0:
            prediction_result = [['Id', 'Category']]
            prediction_result += [[0, predicted_class]]
        else:
            prediction_result += [[int(i), predicted_class]]

        if i%5==0:
            print(str(i)+' / '+str(image_count)+' completed...')

        if image_count != 4977:
            print("You are doing someting wrong!!!")

    except:
        print('ERROR!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        prediction_result += [[int(i), predicted_class]]



In [ ]:
#--------------------------------------------
# Write prediction as CSV
#--------------------------------------------

import csv

print(prediction_result)

f = open('prediction.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(prediction_result)
f.close()


In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))